In [2]:
pip install pandas scikit-learn cohere

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import cohere
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Load dataset
df = pd.read_csv('test.csv')  

X = df['query']
y_true = df['category']

co = cohere.Client('samplekey')

X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.2, random_state=42)

custom_model_pipeline = make_pipeline(
    TfidfVectorizer(),
    LogisticRegression(max_iter=1000)
)

custom_model_pipeline.fit(X_train, y_train)

# Custom model prediction function
def custom_model_predict(query):
    return custom_model_pipeline.predict([query])[0]

# Cohere model prediction function
def cohere_predict(query):
    response = co.classify(
        model='large',  # Use the appropriate model for your use case
        inputs=[query],
        examples=[
            cohere.Example("How do I reset my password?", "IT"),
            cohere.Example("What are the vacation policies?", "HR"),
            cohere.Example("Legal approval for contracts", "Legal"),
            # Add more examples for better accuracy
        ]
    )
    return response.classifications[0].prediction

# Evaluate Cohere model predictions
y_pred_cohere = [cohere_predict(query) for query in X]
cohere_accuracy = accuracy_score(y_true, y_pred_cohere)
print("Cohere Model Accuracy:", cohere_accuracy)
print("Cohere Model Classification Report:")
print(classification_report(y_true, y_pred_cohere))

# Evaluate Custom model predictions
y_pred_custom = [custom_model_predict(query) for query in X]
custom_model_accuracy = accuracy_score(y_true, y_pred_custom)
print("Custom Model Accuracy:", custom_model_accuracy)
print("Custom Model Classification Report:")
print(classification_report(y_true, y_pred_custom))

# Compare results in a DataFrame
comparison_df = pd.DataFrame({
    'Query': X,
    'True Category': y_true,
    'Cohere Prediction': y_pred_cohere,
    'Custom Model Prediction': y_pred_custom
})

import ace_tools as tools; tools.display_dataframe_to_user(name="Comparison of Models", dataframe=comparison_df)

comparison_df.head()  # Display first few rows


ModuleNotFoundError: No module named 'cohere'